In [2]:
import pandas as pd
import numpy as np

import requests

In [4]:
dmp_flora = pd.read_excel("DMP Flora by EcoUnit.xlsx")

In [5]:
df = dmp_flora.drop_duplicates(subset=["EcosUnit", "SciName"])

Can use saved observations instead of re-querying iNaturalist

In [27]:
def get_lb_data_from_soup(soup, sn, symbol):
    #print('Getting Lady Bird data from soup')
    rec = {"SciName": sn, 
           "Symbol": symbol, 
           "Bloom Time:": " ",
           "Jan": 0,
           "Feb": 0,
           "Mar": 0,
           "Apr": 0,
           "May": 0,
           "Jun": 0,
           "Jul": 0,
           "Aug": 0,
           "Sep": 0,
           "Oct": 0,
           "Nov": 0,
           "Dec": 0,
           "Special Value to Native Bees": 0 ,
           "Special Value to Bumble Bees": 0, 
           "Nectar Source:": 0, 
           "Larval Host:": "No",
           "Larval Host(Monarch)": 0,
           "Larval Host(Other)": 0}
    h4s = soup.find_all("h4")
    #print("h4's found: ",h4s)
    for n in h4s:
        #print('\n\n', '####', n, n.string)
        if n.string == "Bloom Information":
            #print("Found Bloom Information")
            sibs = n.find_next_siblings()
            for s in sibs:
                #print(s)
                if s.string == "Bloom Time:":
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    mos = s.next_sibling.split(",")
                    for m in mos:
                        rec[m.strip()] = 1                    
                    rec[s.string] = s.next_sibling
        if n.string == "Benefit":
            sibs = n.find_next_siblings()
            for s in sibs:
                #print(s)
                if s.string == "Nectar Source:":
                    print("found benefit for ", sn, symbol)
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    rec[s.string] = 1
                if s.string == "Larval Host:":
                    print("found benefit for ", sn, symbol)
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    rec[s.string] = s.next_sibling
                    if "onarch" in s.next_sibling:
                        rec["Larval Host(Monarch)"] = 1
                    else: 
                        rec["Larval Host(Other)"] = 1
        if n.string == "Value to Beneficial Insects":
            #print(n.parent.prettify())
            for s in n.find_next_siblings():
                #print(s)
                if s.string: 
                    if (s.string.strip().startswith("Special") and (s.string.find("Honey") == -1)):
                        #print('adding string', s)
                        rec[s.string.replace("\xa0", " ")] = 1
                #print(s.next_sibling)
                if (str(s.next_sibling).strip().startswith("Special") and (str(s.next_sibling).find("Honey") == -1)):
                    #print("adding next sibling", s.next_sibling)
                    rec[str(s.next_sibling.strip().replace("\xa0", " "))] = 1                
    return rec

Run the two above functions once for each species reported, and collect the records into a dataframe

In [ ]:
rec_list = []
for plant in dfp_flora.SciName.values:
    #print(plant)
    page, sn, symbol = get_lb_page_for_sci_name(plant)
    rec_list.append(get_lb_data_from_soup(page, sn, symbol))


In [ ]:
plant_df = pd.DataFrame(rec_list)

Save the species information to an Excel sheet

In [143]:
plant_df.to_excel("LadyBirdData_iNaturalist_expanded.xlsx")

Can use the saved plant info data if it seems current enough

In [8]:
plant_df = pd.read_excel("LadyBirdData_iNaturalist_expanded.xlsx")

Clean up columns

In [9]:
plant_df = plant_df[['SciName', 'Symbol', 'Bloom Time:', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
       'Special Value to Native Bees', 'Special Value to Bumble Bees',
       'Nectar Source:', 'Larval Host:', 'Larval Host(Monarch)',
       'Larval Host(Other)']]

Add the plant characteristics to the observations

In [10]:
mdf =  df.join(plant_df.reset_index().set_index("SciName"), on="SciName")

In [12]:
mdf["habitat_index"] = (mdf["Special Value to Native Bees"] 
     + mdf["Special Value to Bumble Bees"] 
     + mdf["Nectar Source:"] 
     + mdf["Larval Host(Monarch)"] 
     + mdf["Larval Host(Other)"])
mdf["# months with blooms"] = (mdf["Jan"] + mdf["Feb"] + mdf["Mar"] + mdf["Apr"] 
     + mdf["May"] + mdf["Jun"] + mdf["Jul"] + mdf["Aug"] + mdf["Sep"] 
     + mdf["Oct"] + mdf["Nov"] + mdf["Dec"] )
mdf = mdf.drop_duplicates(subset=["EcosUnit", "SciName"])
int_df = mdf[mdf.habitat_index > 0]

Save the data to excel

In [159]:
mdf = mdf[mdf.habitat_index > 0]  # Should be unnecessary, we already filterd to "Interesting?"
#mdf = mdf.drop_duplicates(subset=["EUnit", "Symbol"])

In [14]:
sum_df = int_df.groupby([ 'EcosName', 'EcosUnit' ]).agg({'SciName':'count', 
                                                                             'Special Value to Native Bees':'sum', 
                                                                             'Special Value to Bumble Bees':'sum',
                                                                             'Nectar Source:': "sum", 
                                                                             'Larval Host:': "sum",
                                                                             'Larval Host(Monarch)': "sum", 
                                                                             'Larval Host(Other)': "sum",
                                                                             '# months with blooms': "sum"
                                    })

In [173]:
sum_df.sort_values("common_name")

common_name
EcosName           EUnit SciName                Habitat   Classifica FlorRating GeneralCom      CommType            Acres Shape_Area   observed_on who         Symbol Bloom Time:                         Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec  Special Value to Native Bees Special Value to Bumble Bees Nectar Source: Larval Host:                                       Larval Host(Monarch) Larval Host(Other) habitat_index # months with blooms             
101ST STREET LOT   EWF84 Baptisia alba          Valuable  2          II         Developed       Mowed Areas         2.39  104,237.06   2021-05-24  dicot1      BAAL    Apr , May , Jun , Jul              0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 0.00 1.00                         1.00                         0.00           No                                                 0.00                 0.00               2.00          4.00                            1
RECREATION AREA    EMA15 Bouteloua curtipendula Important 3          III        Eurasian Meadow Forb Meadow         50.24 2,188,342.63 2021-01-06  adriansydor BOCU    Jun , Jul , Aug , Sep , Oct , Nov  0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 1.00 1.00 0.00 0.00                         0.00                         0.00            Green Skipper butterfly, Dotted Skipper butter... 0.00                 1.00               1.00          6.00                            1
                         Eryngium yuccifolium   Important 3          III        Eurasian Meadow Forb Meadow         50.24 2,188,342.63 2021-01-06  adriansydor ERYU    May , Jun , Jul , Aug              0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00 1.00                         0.00                         0.00           No                                                 0.00                 0.00               1.00          4.00                            1
                         Lythrum alatum         Important 3          III        Eurasian Meadow Forb Meadow         50.24 2,188,342.63 2021-01-06  adriansydor LYAL4   May , Jun , Jul , Aug , Sep        0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 1.00                         1.00                         1.00           No                                                 0.00                 0.00               3.00          5.00                            1
                         Mimulus ringens        Important 3          III        Eurasian Meadow Forb Meadow         50.24 2,188,342.63 2021-01-06  adriansydor MIRI    Jun , Jul , Aug , Sep              0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 0.00                         0.00                         0.00            Common Buckeye, Baltimore checkerspots            0.00                 1.00               1.00          4.00                            1
...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   ...
RECREATIONAL AREAS EDA13 Monarda fistulosa      Important 3          II         Eurasian Meadow Tall-grass Meadow   97.27 4,275,279.67 2018-12-30  brandon236  MOBR2   Jun , Jul , Aug , Sep              0.00 0.00 0.00 0.00 0.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00 1.00                         1.00                         0.00           No                                                 0.00                 0.00               2.00          4.00                            4
THUNDERBIRD WOODS  EGV20 Quercus macrocarpa     Critical  4          IV         Forest          Mesic Woodland      80.33 3,499,307.21 2018-09-14  andrewhipp  QUMA2   Mar , Apr , May              

In [16]:
with pd.ExcelWriter("dfp_interesting_units.xlsx") as writer:
    sum_df.to_excel(writer, sheet_name="SummaryData")
    mdf.set_index("EcosUnit").to_excel(writer, sheet_name="DetailData")

In [85]:
df.groupby(['SciName']).count()['common_name']

SciName
Abutilon theophrasti       2
Acalypha rhomboidea        1
Acer negundo              40
Acer nigrum                1
Acer platanoides           3
                          ..
Vitis riparia              8
Xanthium strumarium       12
Yucca filamentosa          1
Zanthoxylum americanum     2
Zizia aurea               31
Name: common_name, Length: 714, dtype: int64

In [88]:
pd.options.display.float_format = '{:,.2f}'.format